# SDWFS AGN Color--Color Plane Plot
## Author: Benjamin Floyd

Generates a publication plot of the SDWFS AGN color--color plane showing both Stern+05 and Donley+12 selections

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join
from matplotlib.lines import Line2D
from matplotlib.patches import Polygon

%matplotlib inline
plt.style.use('seaborn-colorblind')

In [ ]:
# Read in the SDWFS photometric catalog
sdwfs_main = Table.read('Data_Repository/Catalogs/Bootes/SDWFS/ch2v33_sdwfs_2009mar3_apcorr_matched_ap4_Main_v0.4.cat.gz',
                        names=['ID', 'IRAC_RA', 'IRAC_DEC', 'B_APFLUX4', 'R_APFLUX4', 'I_APFLUX4', 'B_APFLUXERR4',
                               'R_APFLUXERR4', 'I_APFLUXERR4', 'B_APMAG4', 'R_APMAG4', 'I_APMAG4', 'B_APMAGERR4',
                               'R_APMAGERR4', 'I_APMAGERR4', 'CH1_APFLUX4', 'CH2_APFLUX4', 'CH3_APFLUX4', 'CH4_APFLUX4',
                               'CH1_APFLUXERR4', 'CH2_APFLUXERR4', 'CH3_APFLUXERR4', 'CH4_APFLUXERR4',
                               'CH1_APFLUXERR4_BROWN', 'CH2_APFLUXERR4_BROWN', 'CH3_APFLUXERR4_BROWN',
                               'CH4_APFLUXERR4_BROWN', 'CH1_APMAG4', 'CH2_APMAG4', 'CH3_APMAG4', 'CH4_APMAG4',
                               'CH1_APMAGERR4', 'CH2_APMAGERR4', 'CH3_APMAGERR4', 'CH4_APMAGERR4',
                               'CH1_APMAGERR4_BROWN', 'CH2_APMAGERR4_BROWN', 'CH3_APMAGERR4_BROWN',
                               'CH4_APMAGERR4_BROWN', 'STARS_COLOR', 'STARS_MORPH', 'CLASS_STAR', 'MBZ_FLAG_4_4_4'],
                        format='ascii')

# Read in the photometric redshift catalog
sdwfs_photz = Table.read('Data_Repository/Catalogs/Bootes/SDWFS/mbz_v0.06_prior_bri12_18p8.cat.gz',
                         names=['ID', 'PHOT_Z', 'col3', 'col4', 'col5', 'col6', 'col7'],
                         format='ascii',
                         include_names=['ID', 'PHOT_Z'])

# Join the two catalogs together
sdwfs_cat = join(sdwfs_main, sdwfs_photz, keys='ID')

In [ ]:
# For convenience, add columns for the two colors
sdwfs_cat['CH1_CH2_COLOR'] = sdwfs_cat['CH1_APMAG4'] - sdwfs_cat['CH2_APMAG4']
sdwfs_cat['CH3_CH4_COLOR'] = sdwfs_cat['CH3_APMAG4'] - sdwfs_cat['CH4_APMAG4']

# And add columns for the log flux ratios
sdwfs_cat['CH3_CH1_LOG_FLUX'] = np.log10(sdwfs_cat['CH3_APFLUX4'] / sdwfs_cat['CH1_APFLUX4'])
sdwfs_cat['CH4_CH2_LOG_FLUX'] = np.log10(sdwfs_cat['CH4_APFLUX4'] / sdwfs_cat['CH2_APFLUX4'])

In [ ]:
# Require SNR cuts of > 5 in all bands for a clean sample
sdwfs_cat = sdwfs_cat[(sdwfs_cat['CH1_APFLUX4'] / sdwfs_cat['CH1_APFLUXERR4'] >= 5) &
                      (sdwfs_cat['CH2_APFLUX4'] / sdwfs_cat['CH2_APFLUXERR4'] >= 5) &
                      (sdwfs_cat['CH3_APFLUX4'] / sdwfs_cat['CH3_APFLUXERR4'] >= 5) &
                      (sdwfs_cat['CH4_APFLUX4'] / sdwfs_cat['CH4_APFLUXERR4'] >= 5)]

In [ ]:
# Make AGN selections using the Stern+05 wedge selection
stern_agn = sdwfs_cat[(sdwfs_cat['CH3_CH4_COLOR'] > 0.6) &
                      (sdwfs_cat['CH1_CH2_COLOR'] > 0.2 * sdwfs_cat['CH3_CH4_COLOR'] + 0.18) &
                      (sdwfs_cat['CH1_CH2_COLOR'] > 2.5 * sdwfs_cat['CH3_CH4_COLOR'] - 3.5)]

donley_agn = sdwfs_cat[(sdwfs_cat['CH3_CH1_LOG_FLUX'] >= 0.08) & (sdwfs_cat['CH4_CH2_LOG_FLUX'] >= 0.15) &
                       (sdwfs_cat['CH4_CH2_LOG_FLUX'] >= 1.21 * sdwfs_cat['CH3_CH1_LOG_FLUX'] - 0.27) &
                       (sdwfs_cat['CH4_CH2_LOG_FLUX'] <= 1.21 * sdwfs_cat['CH3_CH1_LOG_FLUX'] + 0.27) &
                       (sdwfs_cat['CH2_APFLUX4'] > sdwfs_cat['CH1_APFLUX4']) &
                       (sdwfs_cat['CH3_APFLUX4'] > sdwfs_cat['CH2_APFLUX4']) &
                       (sdwfs_cat['CH4_APFLUX4'] > sdwfs_cat['CH3_APFLUX4'])]

In [ ]:
# Identify the IDs of the objects outside the Stern-wedge selection to create a sample of non-AGN.
stern_complement_ids = list(set(sdwfs_cat['ID']) - set(stern_agn['ID']))
non_agn = sdwfs_cat[np.in1d(sdwfs_cat['ID'], stern_complement_ids)]

In [ ]:
# Make plot
fig, ax = plt.subplots()
# Sample hexbins
ax.hexbin(non_agn['CH3_CH4_COLOR'], non_agn['CH1_CH2_COLOR'], gridsize=50, extent=(0., 3., 0., 1.5), cmap='Blues', bins=None, mincnt=1)
ax.hexbin(stern_agn['CH3_CH4_COLOR'], stern_agn['CH1_CH2_COLOR'], gridsize=50, extent=(0., 3., 0., 1.5), cmap='Oranges', bins=None, mincnt=1, alpha=0.6)
ax.hexbin(donley_agn['CH3_CH4_COLOR'], donley_agn['CH1_CH2_COLOR'], gridsize=50, extent=(0., 3., 0., 1.5), cmap='Greens', bins=None, mincnt=1, alpha=0.3)

# Stern wedge bounds
ymin, ymax = ax.get_ylim()
ax.add_patch(Polygon([(0.6, ymax), (0.6, 0.3), (1.6, 0.5), ((ymax + 3.5) / 2.5, ymax)], closed=False))

# Donley Power-law Locus line
# ax.plot()

# Make legend
handles, labels = ax.get_legend_handles_labels()
handles.extend([Line2D([0], [0], marker='h', color='w', markerfacecolor='lightcoral', markeredgecolor='firebrick', markersize=10),
                Line2D([0], [0], marker='h', color='w', markerfacecolor='lightgreen', markeredgecolor='seagreen', markersize=10),
                Line2D([0], [0], marker='h', color='w', markerfacecolor='lightblue', markeredgecolor='steelblue', markersize=10)])
labels.extend(['Stern Wedge AGN', 'Donley Power-Law Locus AGN', 'Non-Active Galaxies'])
ax.legend(handles, labels, frameon=False)

ax.set(xlabel='[5.8] - [8.0] (Vega)', ylabel='[3.6] - [4.5] (Vega)')
fig.savefig('Data_Repository/Project_Data/SPT-IRAGN/Publication_Plots/SDWFS_AGN_color-color_publication.pdf')
plt.show()